In [209]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from scipy.ndimage.filters import gaussian_filter
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import seaborn as sns
%matplotlib inline



In [210]:
spark = SparkSession.builder\
    .appName("demographics")\
    .getOrCreate()

In [211]:
demog = spark.read.csv('/Users/vvviren/Desktop/Demographic_Statistics_By_Zip_Code.csv',header='true')


In [212]:
demog.printSchema()

root
 |-- JURISDICTION NAME: string (nullable = true)
 |-- COUNT PARTICIPANTS: string (nullable = true)
 |-- COUNT FEMALE: string (nullable = true)
 |-- PERCENT FEMALE: string (nullable = true)
 |-- COUNT MALE: string (nullable = true)
 |-- PERCENT MALE: string (nullable = true)
 |-- COUNT GENDER UNKNOWN: string (nullable = true)
 |-- PERCENT GENDER UNKNOWN: string (nullable = true)
 |-- COUNT GENDER TOTAL: string (nullable = true)
 |-- PERCENT GENDER TOTAL: string (nullable = true)
 |-- COUNT PACIFIC ISLANDER: string (nullable = true)
 |-- PERCENT PACIFIC ISLANDER: string (nullable = true)
 |-- COUNT HISPANIC LATINO: string (nullable = true)
 |-- PERCENT HISPANIC LATINO: string (nullable = true)
 |-- COUNT AMERICAN INDIAN: string (nullable = true)
 |-- PERCENT AMERICAN INDIAN: string (nullable = true)
 |-- COUNT ASIAN NON HISPANIC: string (nullable = true)
 |-- PERCENT ASIAN NON HISPANIC: string (nullable = true)
 |-- COUNT WHITE NON HISPANIC: string (nullable = true)
 |-- PERCENT WHI

In [213]:
demog.select(demog['count participants']).show()

+------------------+
|count participants|
+------------------+
|                44|
|                35|
|                 1|
|                 0|
|                 2|
|                 6|
|                 1|
|                 2|
|                 0|
|                 3|
|                 0|
|                 8|
|                 0|
|                17|
|                 0|
|                 3|
|                 0|
|                 0|
|                 0|
|                 1|
+------------------+
only showing top 20 rows



In [214]:
demog = demog.filter(demog['count participants'] != '0')

In [215]:
demog.select(demog['jurisdiction name']).show()

+-----------------+
|jurisdiction name|
+-----------------+
|            10001|
|            10002|
|            10003|
|            10005|
|            10006|
|            10007|
|            10009|
|            10011|
|            10013|
|            10016|
|            10018|
|            10022|
|            10023|
|            10024|
|            10025|
|            10027|
|            10029|
|            10032|
|            10036|
|            10038|
+-----------------+
only showing top 20 rows



In [216]:
cols=[]
for c in demog.columns:
    if 'COUNT' in c:
        cols.append(c)

cols.remove('COUNT PARTICIPANTS')
cols

['COUNT FEMALE',
 'COUNT MALE',
 'COUNT GENDER UNKNOWN',
 'COUNT GENDER TOTAL',
 'COUNT PACIFIC ISLANDER',
 'COUNT HISPANIC LATINO',
 'COUNT AMERICAN INDIAN',
 'COUNT ASIAN NON HISPANIC',
 'COUNT WHITE NON HISPANIC',
 'COUNT BLACK NON HISPANIC',
 'COUNT OTHER ETHNICITY',
 'COUNT ETHNICITY UNKNOWN',
 'COUNT ETHNICITY TOTAL',
 'COUNT PERMANENT RESIDENT ALIEN',
 'COUNT US CITIZEN',
 'COUNT OTHER CITIZEN STATUS',
 'COUNT CITIZEN STATUS UNKNOWN',
 'COUNT CITIZEN STATUS TOTAL',
 'COUNT RECEIVES PUBLIC ASSISTANCE',
 'COUNT NRECEIVES PUBLIC ASSISTANCE',
 'COUNT PUBLIC ASSISTANCE UNKNOWN',
 'COUNT PUBLIC ASSISTANCE TOTAL']

In [217]:
demog = demog.drop(*cols)

In [218]:
demog.columns

['JURISDICTION NAME',
 'COUNT PARTICIPANTS',
 'PERCENT FEMALE',
 'PERCENT MALE',
 'PERCENT GENDER UNKNOWN',
 'PERCENT GENDER TOTAL',
 'PERCENT PACIFIC ISLANDER',
 'PERCENT HISPANIC LATINO',
 'PERCENT AMERICAN INDIAN',
 'PERCENT ASIAN NON HISPANIC',
 'PERCENT WHITE NON HISPANIC',
 'PERCENT BLACK NON HISPANIC',
 'PERCENT OTHER ETHNICITY',
 'PERCENT ETHNICITY UNKNOWN',
 'PERCENT ETHNICITY TOTAL',
 'PERCENT PERMANENT RESIDENT ALIEN',
 'PERCENT US CITIZEN',
 'PERCENT OTHER CITIZEN STATUS',
 'PERCENT CITIZEN STATUS UNKNOWN',
 'PERCENT CITIZEN STATUS TOTAL',
 'PERCENT RECEIVES PUBLIC ASSISTANCE',
 'PERCENT NRECEIVES PUBLIC ASSISTANCE',
 'PERCENT PUBLIC ASSISTANCE UNKNOWN',
 'PERCENT PUBLIC ASSISTANCE TOTAL']

In [219]:
fp = spark.read.csv('/Users/vvviren/Desktop/Film_Permits.csv',header='true')

In [220]:
fp.printSchema()

root
 |-- EventID: string (nullable = true)
 |-- EventType: string (nullable = true)
 |-- StartDateTime: string (nullable = true)
 |-- EndDateTime: string (nullable = true)
 |-- EnteredOn: string (nullable = true)
 |-- EventAgency: string (nullable = true)
 |-- ParkingHeld: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- CommunityBoard(s): string (nullable = true)
 |-- PolicePrecinct(s): string (nullable = true)
 |-- Category: string (nullable = true)
 |-- SubCategoryName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- ZipCode(s): string (nullable = true)



In [221]:
cols = ['EventId', 'EventAgency', 'CommunityBoard', 'PolicePrecint', 'Country']
fp.drop(*cols)

DataFrame[EventType: string, StartDateTime: string, EndDateTime: string, EnteredOn: string, ParkingHeld: string, Borough: string, CommunityBoard(s): string, PolicePrecinct(s): string, Category: string, SubCategoryName: string, ZipCode(s): string]

In [222]:
fp.select(fp['ZipCode(s)']).show()

+--------------------+
|          ZipCode(s)|
+--------------------+
|               10012|
|        10034, 10463|
|               11378|
|               11201|
|        10001, 10121|
|        11101, 11222|
|               11217|
|               10036|
|               10462|
|               10014|
|        10036, 10105|
|        10014, 11101|
|        11203, 11218|
|        10001, 10121|
|               11219|
|               10001|
|               10036|
|               11201|
|               11104|
|10017, 10022, 101...|
+--------------------+
only showing top 20 rows



In [223]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split


fp=fp.withColumn('tmp', split('ZipCode(s)', ', '))
                                               
fp=fp.withColumn('zips', explode('tmp'))

In [224]:
fp.select('EventID', 'zips').show()

+-------+-----+
|EventID| zips|
+-------+-----+
| 446040|10012|
| 446168|10034|
| 446168|10463|
| 186438|11378|
| 445255|11201|
| 128794|10001|
| 128794|10121|
|  43547|11101|
|  43547|11222|
|  66846|11217|
| 104342|10036|
| 244863|10462|
| 446379|10014|
| 446359|10036|
| 446359|10105|
| 203743|10014|
| 203743|11101|
| 446069|11203|
| 446069|11218|
| 445165|10001|
+-------+-----+
only showing top 20 rows



In [225]:
left_join = fp.join(demog, fp['zips'] == demog['JURISDICTION NAME'],how='left') # Could also use 'left_outer'
left_join.select('percent male').show()

+------------+
|percent male|
+------------+
|        null|
|        null|
|        0.44|
|        null|
|        0.45|
|         0.5|
|        null|
|           1|
|        null|
|        null|
|           0|
|        0.33|
|        null|
|           0|
|        null|
|        null|
|           1|
|         0.2|
|        0.13|
|         0.5|
+------------+
only showing top 20 rows



In [226]:
left_join.columns

['EventID',
 'EventType',
 'StartDateTime',
 'EndDateTime',
 'EnteredOn',
 'EventAgency',
 'ParkingHeld',
 'Borough',
 'CommunityBoard(s)',
 'PolicePrecinct(s)',
 'Category',
 'SubCategoryName',
 'Country',
 'ZipCode(s)',
 'tmp',
 'zips',
 'JURISDICTION NAME',
 'COUNT PARTICIPANTS',
 'PERCENT FEMALE',
 'PERCENT MALE',
 'PERCENT GENDER UNKNOWN',
 'PERCENT GENDER TOTAL',
 'PERCENT PACIFIC ISLANDER',
 'PERCENT HISPANIC LATINO',
 'PERCENT AMERICAN INDIAN',
 'PERCENT ASIAN NON HISPANIC',
 'PERCENT WHITE NON HISPANIC',
 'PERCENT BLACK NON HISPANIC',
 'PERCENT OTHER ETHNICITY',
 'PERCENT ETHNICITY UNKNOWN',
 'PERCENT ETHNICITY TOTAL',
 'PERCENT PERMANENT RESIDENT ALIEN',
 'PERCENT US CITIZEN',
 'PERCENT OTHER CITIZEN STATUS',
 'PERCENT CITIZEN STATUS UNKNOWN',
 'PERCENT CITIZEN STATUS TOTAL',
 'PERCENT RECEIVES PUBLIC ASSISTANCE',
 'PERCENT NRECEIVES PUBLIC ASSISTANCE',
 'PERCENT PUBLIC ASSISTANCE UNKNOWN',
 'PERCENT PUBLIC ASSISTANCE TOTAL']

In [227]:
from pyspark.sql.functions import col
import pyspark.sql.functions as F

cntfp = left_join.groupby(left_join.zips).count()
sortcntfp = cntfp.sort(col('count').desc())
sortcntfp = sortcntfp.filter(col('count')>1100)
sortcntfp.show()

+-----+-----+
| zips|count|
+-----+-----+
|11222| 7271|
|11101| 5694|
|10036| 4174|
|10019| 3969|
|10001| 3165|
|10013| 2998|
|10011| 2765|
|11201| 2516|
|10003| 2459|
|10023| 2401|
|10014| 2017|
|10012| 1976|
|11217| 1922|
|10002| 1908|
|11211| 1762|
|11106| 1581|
|11249| 1566|
|10022| 1352|
|10010| 1246|
|11237| 1188|
+-----+-----+
only showing top 20 rows



In [228]:
from pyspark.sql.functions import col

filterfp = left_join.join(sortcntfp, left_join['zips']==sortcntfp['zips']).drop(sortcntfp.zips)
filterfp.select('PERCENT BLACK NON HISPANIC').distinct().show()

+--------------------------+
|PERCENT BLACK NON HISPANIC|
+--------------------------+
|                      0.03|
|                         0|
|                      null|
|                      0.45|
|                      0.33|
|                       0.5|
|                      0.48|
|                      0.43|
+--------------------------+



In [229]:
from pyspark.sql.types import DoubleType
filterfp = filterfp.withColumn("percent_black_non_hispanic", filterfp["PERCENT BLACK NON HISPANIC"].cast(DoubleType()))

filterfp.filter(col('percent_black_non_hispanic') >0).select('zips', 'percent_black_non_hispanic').distinct().show()

+-----+--------------------------+
| zips|percent_black_non_hispanic|
+-----+--------------------------+
|10011|                      0.33|
|10001|                      0.48|
|10023|                      0.43|
|11201|                      0.45|
|10036|                       0.5|
|11211|                      0.03|
+-----+--------------------------+



In [230]:
df = sortcntfp.join(demog, sortcntfp['zips'] == demog['JURISDICTION NAME'],how='left') # Could also use 'left_outer'

In [245]:
from pyspark.sql.types import DoubleType
df = df.withColumn("percent_us_citizen", df["PERCENT US CITIZEN"].cast(DoubleType()))
df = df.withColumn("percent_other_citizen", df["PERCENT OTHER CITIZEN STATUS"].cast(DoubleType()))
df = df.withColumn("percent_unknown_citizen", df["PERCENT CITIZEN STATUS UNKNOWN"].cast(DoubleType()))
df = df.withColumn("percent_hispanic_latino", df["PERCENT HISPANIC LATINO"].cast(DoubleType()))
df = df.withColumn("percent_american_indian", df["PERCENT AMERICAN INDIAN"].cast(DoubleType()))
df = df.withColumn("percent_male", df["PERCENT MALE"].cast(DoubleType()))
df = df.withColumn("percent_female", df["PERCENT FEMALE"].cast(DoubleType()))
df = df.withColumn("percent_asian_non_hispanic", df['PERCENT ASIAN NON HISPANIC'].cast(DoubleType()))
df = df.withColumn("percent_white_non_hispanic", df['PERCENT WHITE NON HISPANIC'].cast(DoubleType()))
df = df.withColumn("percent_black_non_hispanic", df['PERCENT BLACK NON HISPANIC'].cast(DoubleType()))
df = df.withColumn("percent_ethinicity_unknown", df['PERCENT ETHNICITY UNKNOWN'].cast(DoubleType()))


df.sort(col('percent_us_citizen').desc()).select('zips','count','percent_us_citizen').show()
df.sort(col('percent_hispanic_latino').desc()).select('zips','count','percent_hispanic_latino').show()
df.sort(col('percent_male').desc()).select('zips','count','percent_male').show()

+-----+-----+------------------+
| zips|count|percent_us_citizen|
+-----+-----+------------------+
|10013| 2998|               1.0|
|10036| 4174|               1.0|
|11211| 1762|               1.0|
|10011| 2765|               1.0|
|10023| 2401|               1.0|
|11101| 5694|               1.0|
|10022| 1352|               1.0|
|10003| 2459|               1.0|
|11201| 2516|               1.0|
|10001| 3165|              0.95|
|10002| 1908|              0.94|
|10018| 1154|              0.67|
|11385| 1162|              null|
|10019| 3969|              null|
|10012| 1976|              null|
|11106| 1581|              null|
|10014| 2017|              null|
|11217| 1922|              null|
|11249| 1566|              null|
|10010| 1246|              null|
+-----+-----+------------------+
only showing top 20 rows

+-----+-----+-----------------------+
| zips|count|percent_hispanic_latino|
+-----+-----+-----------------------+
|10023| 2401|                   0.43|
|10001| 3165|                 

In [233]:
df.stat.corr('count', 'percent_us_citizen')


0.1685760188209232

In [234]:
df.stat.corr('count', 'percent_other_citizen')

nan

In [235]:
df.stat.corr('count', 'percent_unknown_citizen')

nan

In [236]:
df.stat.corr('count', 'percent_hispanic_latino')

0.051923800170978175

In [237]:
df.stat.corr('count', 'percent_american_indian')

nan

In [238]:
df.stat.corr('count', 'percent_male')

0.32503401399349197

In [239]:
df.stat.corr('count', 'percent_us_citizen')

0.1685760188209232

In [240]:
df.stat.corr('count', 'percent_us_citizen')

0.1685760188209232

In [241]:
df.stat.corr('count', 'percent_white_non_hispanic')

-0.11742141292817126

In [242]:
df.stat.corr('count', 'percent_black_non_hispanic')

0.1892730768700985

In [246]:
df.stat.corr('count', 'percent_ethinicity_unknown')

nan